# Hyperparameter Optimization with Hyperopt
Here, we try out the hyperopt package and try to use a simple scikitlearn classifier and tune its hyperparameter using the hyperopt package.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df_train = pd.read_table('data/labeledTrainData.tsv', sep='\t')
df_test = pd.read_table('data/testData.tsv', sep='\t')

# Add a column for the score (out of 10)
df_train['score'] = df_train['id'].map(lambda x: int(x.split('_')[1]))
df_test['score'] = df_test['id'].map(lambda x: int(x.split('_')[1]))

# Add a column for the class, 1 for positive, 0 for negative
df_train['class'] = df_train['score'].map(lambda x: int(x > 5))
df_test['class'] = df_test['score'].map(lambda x: int(x > 5))

# Sort the df_test by class (it will need to be sorted later)
df_test = df_test.sort_values('class', ascending=False)

In [3]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words('english')

# Function for removing stopwords from a string
def removeStopwords(s):
    return ' '.join([word for word in s.split() if word not in cachedStopWords])

# Function for cleaning the reviews
def cleanText(s):
    s = s.str.lower()                         # Convert to lowercase
    s = s.str.replace(r'<.*?>', ' ')          # Remove HTML characters
    s = s.str.replace('\'', '')               # Remove single quotes ' 
    s = s.str.replace('-', '')                # Remove dashes -
    s = s.str.replace(r'[^a-zA-Z]', ' ')      # Remove non alpha characters
    s = s.map(lambda s: ' '.join(s.split()))  # Remove extra whitespae
    s = s.str.strip()                         # Remove whitespace at start and end
    s = s.apply(lambda x: removeStopwords(x)) # Remove stopwords
    return s
 
# Clean the reviews
df_train['review'] = cleanText(df_train['review'])
df_test['review'] = cleanText(df_test['review'])

In [4]:
# Separate into training and test data
X_train = df_train['review']
y_train = df_train['class']
X_test = df_test['review']
y_test = df_test['class']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection.univariate_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline

# The model
vectorizer = TfidfVectorizer(strip_accents='unicode')
kbest = SelectPercentile(chi2)
clf = SGDClassifier(loss='modified_huber', penalty='elasticnet')

pipe = Pipeline([('vec', vectorizer),
                 ('kbest', kbest),
                 ('clf', clf)])

In [6]:
from sklearn import metrics
pipe.fit(X_train, y_train)
metrics.roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1])

/usr/local/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.95221796479999998

In [7]:
from hyperopt import hp

# Parameter search space
space = {}
# One of (1,1), (1,2), or (1,3)
space['vec__ngram_range'] = hp.choice('vec__ngram_range', [(1,1), (1,2), (1,3)])
# Uniform between 0.7 and 1
space['vec__max_df'] = hp.uniform('vec__max_df', 0.7, 1.0)
# One of True or False
space['vec__sublinear_tf'] = hp.choice('vec__sublinear_tf', [True, False])
# Random number between 50 and 100
space['kbest__percentile'] = hp.uniform('kbest__percentile', 50, 100)
# Random number between 0 and 1
space['clf__l1_ratio'] = hp.uniform('clf__l1_ratio', 0.0, 1.0)
# Log-uniform between 1e-9 and 1e-4
space['clf__alpha'] = hp.loguniform('clf__alpha', -9*np.log(10), -4*np.log(10))
# Random integer in 20:5:80
space['clf__n_iter'] = 20 + 5*hp.randint('clf__n_iter', 12)

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

def objective(params):
    pipe.set_params(**params)
    shuffle = KFold(n_splits=2, shuffle=True)
    score = cross_val_score(pipe, X_train, y_train, cv=shuffle, scoring='roc_auc', n_jobs=1)
    return 1-score.mean()

In [9]:
from hyperopt import fmin, tpe, Trials

# The Trials object will store details of each iteration
trials = Trials()

# Run the hyperparameter search using the tpe algorithm
best = fmin(objective,
            space,
            algo=tpe.suggest,
            max_evals=1,
            trials=trials)

/usr/local/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [16]:
trials.best_trial

{'book_time': datetime.datetime(2018, 6, 22, 11, 16, 30, 591000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'clf__alpha': [0],
   'clf__l1_ratio': [0],
   'clf__n_iter': [0],
   'kbest__percentile': [0],
   'vec__max_df': [0],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'tid': 0,
  'vals': {'clf__alpha': [5.0397470112471446e-08],
   'clf__l1_ratio': [0.42820403440560584],
   'clf__n_iter': [4],
   'kbest__percentile': [53.037777369405795],
   'vec__max_df': [0.9630123363033276],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2018, 6, 22, 11, 16, 38, 739000),
 'result': {'loss': 0.06243755609503343, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 0,
 'version': 0}

In [55]:
trials.insert_trial_doc(doc=doc)

2

In [56]:
trials.trials

[{'book_time': datetime.datetime(2018, 6, 22, 11, 16, 30, 591000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'clf__alpha': [0],
    'clf__l1_ratio': [0],
    'clf__n_iter': [0],
    'kbest__percentile': [0],
    'vec__max_df': [0],
    'vec__ngram_range': [0],
    'vec__sublinear_tf': [0]},
   'tid': 0,
   'vals': {'clf__alpha': [5.0397470112471446e-08],
    'clf__l1_ratio': [0.42820403440560584],
    'clf__n_iter': [4],
    'kbest__percentile': [53.037777369405795],
    'vec__max_df': [0.9630123363033276],
    'vec__ngram_range': [0],
    'vec__sublinear_tf': [0]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2018, 6, 22, 11, 16, 38, 739000),
  'result': {'loss': 0.06243755609503343, 'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0}]

In [54]:
import datetime
doc = {'book_time': datetime.datetime(2017, 6, 22, 11, 16, 30, 591000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'clf__alpha': [0],
   'clf__l1_ratio': [0],
   'clf__n_iter': [0],
   'kbest__percentile': [0],
   'vec__max_df': [0],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'tid': 2,
  'vals': {'clf__alpha': [5.0397470112471446e-08],
   'clf__l1_ratio': [0.42820403440560584],
   'clf__n_iter': [4],
   'kbest__percentile': [53.037777369405795],
   'vec__max_df': [0.9630123363033276],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2017, 6, 22, 11, 16, 38, 739000),
 'result': {'loss': 0.05243755609503343, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 2,
 'version': 0}

In [52]:
trials.new_trial_ids(N=19)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [67]:
a = trials.next

In [77]:
trials.trials.append(doc)

In [78]:
trials.trials

[{'book_time': datetime.datetime(2018, 6, 22, 11, 16, 30, 591000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'clf__alpha': [0],
    'clf__l1_ratio': [0],
    'clf__n_iter': [0],
    'kbest__percentile': [0],
    'vec__max_df': [0],
    'vec__ngram_range': [0],
    'vec__sublinear_tf': [0]},
   'tid': 0,
   'vals': {'clf__alpha': [5.0397470112471446e-08],
    'clf__l1_ratio': [0.42820403440560584],
    'clf__n_iter': [4],
    'kbest__percentile': [53.037777369405795],
    'vec__max_df': [0.9630123363033276],
    'vec__ngram_range': [0],
    'vec__sublinear_tf': [0]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2018, 6, 22, 11, 16, 38, 739000),
  'result': {'loss': 0.06243755609503343, 'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0},
 {'book_time': datetime.datetime(2017, 6, 22, 11, 16, 30, 591000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'id

In [79]:
trials.best_trial

{'book_time': datetime.datetime(2017, 6, 22, 11, 16, 30, 591000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'clf__alpha': [0],
   'clf__l1_ratio': [0],
   'clf__n_iter': [0],
   'kbest__percentile': [0],
   'vec__max_df': [0],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'tid': 2,
  'vals': {'clf__alpha': [5.0397470112471446e-08],
   'clf__l1_ratio': [0.42820403440560584],
   'clf__n_iter': [4],
   'kbest__percentile': [53.037777369405795],
   'vec__max_df': [0.9630123363033276],
   'vec__ngram_range': [0],
   'vec__sublinear_tf': [0]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2017, 6, 22, 11, 16, 38, 739000),
 'result': {'loss': 0.05243755609503343, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 2,
 'version': 0}